In [5]:
import os
import time
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

# --- 1. CONFIGURATION PROPRE ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
IMG_DIR = r"C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_train"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Scaler moderne
scaler = torch.amp.GradScaler('cuda') 

print(f"🛡️ PROTOCOLE ANTI-OVERFITTING ACTIVÉ SUR : {DEVICE}")

# --- 2. DATASET AVEC AUGMENTATION FORTE ---
csv_path = os.path.join(PROJECT_ROOT, "data", "raw")
df_x = pd.read_csv(os.path.join(csv_path, "X_train_update.csv"), index_col=0)
df_y = pd.read_csv(os.path.join(csv_path, "Y_train_CVw08PX.csv"), index_col=0)
df = pd.merge(df_x, df_y, left_index=True, right_index=True)
df['path'] = df.apply(lambda x: os.path.join(IMG_DIR, f"image_{x['imageid']}_product_{x['productid']}.jpg"), axis=1)

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['prdtypecode'])
NUM_CLASSES = len(le.classes_)

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)

class RobustDataset(Dataset):
    def __init__(self, df, mode='train'):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        
        # AUGMENTATION "HEAVY" (Empêche le par-cœur)
        self.transform_train = transforms.Compose([
            transforms.Resize((256, 256)),      
            transforms.RandomCrop(224),         
            transforms.RandomHorizontalFlip(p=0.5), 
            transforms.RandomRotation(15),      
            transforms.ColorJitter(brightness=0.2, contrast=0.2), 
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        # VALIDATION STRICTE
        self.transform_val = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    
    def __len__(self): return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try: img = Image.open(row['path']).convert("RGB")
        except: img = Image.new('RGB', (224, 224), (0, 0, 0))
        
        if self.mode == 'train':
            return self.transform_train(img), torch.tensor(row['label_encoded'], dtype=torch.long)
        else:
            return self.transform_val(img), torch.tensor(row['label_encoded'], dtype=torch.long)

train_loader = DataLoader(RobustDataset(train_df, 'train'), batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(RobustDataset(val_df, 'val'), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# --- 3. ARCHITECTURE HYBRIDE ---
print("🔧 Construction du modèle...")
resnet = models.resnet50(weights="IMAGENET1K_V1") 

# Tête Robuste
class LegendMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048), nn.BatchNorm1d(2048), nn.GELU(), nn.Dropout(0.3), 
            nn.Linear(2048, 1024), nn.BatchNorm1d(1024), nn.GELU(), nn.Dropout(0.3),
            nn.Linear(1024, 512),  nn.BatchNorm1d(512),  nn.GELU(), nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    def forward(self, x): return self.net(x)

resnet.fc = LegendMLP(2048, NUM_CLASSES)
model = resnet.to(DEVICE)

# DÉBLOCAGE PARTIEL
for param in model.parameters():
    param.requires_grad = False 

for name, param in model.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True

print("🔓 Déblocage : Layer 3, 4 et Tête.")

# --- 4. OPTIMISEUR ET ENTRAINEMENT ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=1e-2)

# CORRECTION ICI : suppression de verbose=True
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=1)

print("🔥 Démarrage Entraînement 'Generalization' (Objectif 75-80%)...")
best_f1 = 0.0

for epoch in range(15):
    model.train()
    loss_ep = 0.0
    t0 = time.time()
    
    for i, (imgs, lbls) in enumerate(train_loader):
        imgs, lbls = imgs.to(DEVICE), lbls.to(DEVICE)
        
        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):
            out = model(imgs)
            loss = criterion(out, lbls)
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loss_ep += loss.item()
        
        if i % 100 == 0:
            print(f"   ⏳ Ep {epoch+1} | Batch {i} | Loss Train: {loss.item():.4f}", end="\r")

    # Validation
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for imgs, lbls in val_loader:
            imgs = imgs.to(DEVICE)
            with torch.amp.autocast('cuda'):
                out = model(imgs)
            _, p = torch.max(out, 1)
            preds.extend(p.cpu().numpy())
            targets.extend(lbls.cpu().numpy())
    
    val_f1 = f1_score(targets, preds, average='weighted')
    current_lr = optimizer.param_groups[0]['lr']
    duree = time.time() - t0
    
    print(f"\n✅ FIN EP {epoch+1} | Time: {duree:.0f}s | F1 Val: {val_f1:.4f} | LR: {current_lr:.1e}")
    
    scheduler.step(val_f1)
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "livrable_model_resnet_final_clean.pth"))
        
        meta_data = {
            "model_name": "ResNet50 Robust (Anti-Overfit)", 
            "score_f1": val_f1,
            "class_mapping": {int(i): str(c) for i, c in enumerate(le.classes_)}
        }
        with open(os.path.join(OUTPUT_DIR, "livrable_resnet_reborn_metadata.json"), 'w') as f:
            json.dump(meta_data, f, indent=4)
            
        print(f"   💾 Nouveau Champion (Sain) Sauvegardé : {val_f1:.4f}")

print(f"🏆 Entraînement terminé. Score final 'Honnête' : {best_f1:.4f}")

🛡️ PROTOCOLE ANTI-OVERFITTING ACTIVÉ SUR : cuda
🔧 Construction du modèle...
🔓 Déblocage : Layer 3, 4 et Tête.
🔥 Démarrage Entraînement 'Generalization' (Objectif 75-80%)...
   ⏳ Ep 1 | Batch 2100 | Loss Train: 1.6883
✅ FIN EP 1 | Time: 623s | F1 Val: 0.5649 | LR: 1.0e-04
   💾 Nouveau Champion (Sain) Sauvegardé : 0.5649
   ⏳ Ep 2 | Batch 2100 | Loss Train: 1.3254
✅ FIN EP 2 | Time: 625s | F1 Val: 0.5927 | LR: 1.0e-04
   💾 Nouveau Champion (Sain) Sauvegardé : 0.5927
   ⏳ Ep 3 | Batch 2100 | Loss Train: 1.2172
✅ FIN EP 3 | Time: 637s | F1 Val: 0.6095 | LR: 1.0e-04
   💾 Nouveau Champion (Sain) Sauvegardé : 0.6095
   ⏳ Ep 4 | Batch 2100 | Loss Train: 1.1664
✅ FIN EP 4 | Time: 675s | F1 Val: 0.6117 | LR: 1.0e-04
   💾 Nouveau Champion (Sain) Sauvegardé : 0.6117
   ⏳ Ep 5 | Batch 2100 | Loss Train: 0.9467
✅ FIN EP 5 | Time: 652s | F1 Val: 0.6292 | LR: 1.0e-04
   💾 Nouveau Champion (Sain) Sauvegardé : 0.6292
   ⏳ Ep 6 | Batch 2100 | Loss Train: 0.7080
✅ FIN EP 6 | Time: 646s | F1 Val: 0.6151 | 

In [2]:
print("jai eu un bug de kernel donc je réimporte tout ici et je fais un test de num worker pour vérifier ce qui marche ou pas ")


jai eu un bug de kernel donc je réimporte tout ici et je fais un test de num worker pour vérifier ce qui marche ou pas 


In [ ]:
import os
import time
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# --- CONFIGURATION EXPRESS ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
IMG_DIR = r"C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_train"
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"🏎️ DÉBUT DU BENCHMARK SUR : {DEVICE}")

# --- RECONSTRUCTION RAPIDE DU DATASET ---
# (Obligatoire pour que le test fonctionne seul)
csv_path = os.path.join(PROJECT_ROOT, "data", "raw")
df_x = pd.read_csv(os.path.join(csv_path, "X_train_update.csv"), index_col=0)
df_y = pd.read_csv(os.path.join(csv_path, "Y_train_CVw08PX.csv"), index_col=0)
df = pd.merge(df_x, df_y, left_index=True, right_index=True)
df['path'] = df.apply(lambda x: os.path.join(IMG_DIR, f"image_{x['imageid']}_product_{x['productid']}.jpg"), axis=1)

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['prdtypecode'])
train_df, _ = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)

class RobustDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
        # Transformation standard pour le test
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try: img = Image.open(row['path']).convert("RGB")
        except: img = Image.new('RGB', (224, 224), (0, 0, 0))
        return self.transform(img), torch.tensor(row['label_encoded'], dtype=torch.long)

# --- LE TEST DE VITESSE ---
workers_to_test = [0, 1, 2, 4] # On teste ces valeurs

dataset_test = RobustDataset(train_df)
best_worker = 0
best_time = 9999.0

print("-" * 60)
for w in workers_to_test:
    print(f"\n🧪 TEST num_workers = {w} ...")
    
    try:
        # On crée un loader temporaire
        loader = DataLoader(
            dataset_test, 
            batch_size=BATCH_SIZE, 
            shuffle=True, 
            num_workers=w, 
            pin_memory=True, 
            persistent_workers=(w > 0) # Optimisation si > 0
        )
        
        t0 = time.time()
        # On charge 15 batchs pour voir
        for i, _ in enumerate(loader):
            if i >= 15: break
            print(f"   > Batch {i+1}/15 chargé...", end="\r")
            
        duree = time.time() - t0
        speed = duree / 15
        
        print(f"✅ SUCCÈS pour Workers={w} | Temps Total: {duree:.2f}s | Vitesse: {speed:.3f} s/batch")
        
        if speed < best_time:
            best_time = speed
            best_worker = w
            
    except Exception as e:
        print(f"❌ ÉCHEC pour Workers={w} (Erreur ou Freeze)")
    except KeyboardInterrupt:
        print(f"🛑 STOP MANUEL pour Workers={w}")
        break 

print("-" * 60)
print(f"🏆 VAINQUEUR : num_workers = {best_worker}")

🏎️ DÉBUT DU BENCHMARK SUR : cuda
------------------------------------------------------------

🧪 TEST num_workers = 0 ...
✅ SUCCÈS pour Workers=0 | Temps Total: 2.21s | Vitesse: 0.147 s/batch

🧪 TEST num_workers = 1 ...


In [2]:
import os
import time
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

# --- 1. CONFIGURATION ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
IMG_DIR = r"C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_train"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = torch.amp.GradScaler('cuda')

print(f"🚀 RELANCE TOTALE (EXTENSION 30 EPOQUES) SUR : {DEVICE}")

# --- 2. RECONSTRUCTION DES DONNÉES (Obligatoire après redémarrage) ---
print("📂 Rechargement du Dataset...")
csv_path = os.path.join(PROJECT_ROOT, "data", "raw")
df_x = pd.read_csv(os.path.join(csv_path, "X_train_update.csv"), index_col=0)
df_y = pd.read_csv(os.path.join(csv_path, "Y_train_CVw08PX.csv"), index_col=0)
df = pd.merge(df_x, df_y, left_index=True, right_index=True)
df['path'] = df.apply(lambda x: os.path.join(IMG_DIR, f"image_{x['imageid']}_product_{x['productid']}.jpg"), axis=1)

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['prdtypecode'])
NUM_CLASSES = len(le.classes_)

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)

# Définition de la classe Dataset (Celle qui manquait)
class RobustDataset(Dataset):
    def __init__(self, df, mode='train'):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        
        # AUGMENTATION "HEAVY"
        self.transform_train = transforms.Compose([
            transforms.Resize((256, 256)),      
            transforms.RandomCrop(224),         
            transforms.RandomHorizontalFlip(p=0.5), 
            transforms.RandomRotation(15),      
            transforms.ColorJitter(brightness=0.2, contrast=0.2), 
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        # VALIDATION STRICTE
        self.transform_val = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    
    def __len__(self): return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try: img = Image.open(row['path']).convert("RGB")
        except: img = Image.new('RGB', (224, 224), (0, 0, 0))
        
        if self.mode == 'train':
            return self.transform_train(img), torch.tensor(row['label_encoded'], dtype=torch.long)
        else:
            return self.transform_val(img), torch.tensor(row['label_encoded'], dtype=torch.long)

# Configuration Safe (Workers=0) mais optimisée (Pin Memory)
train_loader = DataLoader(RobustDataset(train_df, 'train'), batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(RobustDataset(val_df, 'val'), batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# --- 3. RECONSTRUCTION DU MODÈLE ---
print("🔧 Reconstruction de l'architecture...")
resnet = models.resnet50(weights=None) # Pas besoin des poids ImageNet, on va charger les nôtres

class LegendMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048), nn.BatchNorm1d(2048), nn.GELU(), nn.Dropout(0.3), 
            nn.Linear(2048, 1024), nn.BatchNorm1d(1024), nn.GELU(), nn.Dropout(0.3),
            nn.Linear(1024, 512),  nn.BatchNorm1d(512),  nn.GELU(), nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    def forward(self, x): return self.net(x)

resnet.fc = LegendMLP(2048, NUM_CLASSES)
model = resnet.to(DEVICE)

# --- 4. CHARGEMENT DU CHECKPOINT (CRUCIAL) ---
print("📥 Chargement de ton champion (Epoque 15)...")
checkpoint_path = os.path.join(OUTPUT_DIR, "livrable_model_resnet_final_clean.pth")

if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path))
    print("✅ Poids chargés avec succès ! On reprend l'entraînement.")
else:
    print("❌ ERREUR CRITIQUE : Le fichier .pth n'existe pas !")
    print("   -> Assure-toi d'avoir bien fini les 15 premières époques.")
    # On raise une erreur pour ne pas entraîner un modèle vide par erreur
    raise FileNotFoundError("Modèle 'clean' introuvable.")

# Optimiseur et Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-2) # On force 1e-5 direct
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=1)

# Init best_f1 avec une valeur basse pour être sûr de sauvegarder si ça monte
best_f1 = 0.60 

# --- 5. BOUCLE D'EXTENSION ---
print(f"🔥 Démarrage Extension (Epoques 16 -> 45) | LR Fixé à: 1.0e-05")

for epoch in range(15, 45):
    model.train()
    loss_ep = 0.0
    t0 = time.time()
    
    for i, (imgs, lbls) in enumerate(train_loader):
        imgs, lbls = imgs.to(DEVICE, non_blocking=True), lbls.to(DEVICE, non_blocking=True)
        
        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):
            out = model(imgs)
            loss = criterion(out, lbls)
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loss_ep += loss.item()
        
        if i % 100 == 0:
            print(f"   ⏳ Ep {epoch+1} | Batch {i} | Loss Train: {loss.item():.4f}", end="\r")

    # Validation
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for imgs, lbls in val_loader:
            imgs = imgs.to(DEVICE, non_blocking=True)
            with torch.amp.autocast('cuda'):
                out = model(imgs)
            _, p = torch.max(out, 1)
            preds.extend(p.cpu().numpy())
            targets.extend(lbls.cpu().numpy())
    
    val_f1 = f1_score(targets, preds, average='weighted')
    current_lr = optimizer.param_groups[0]['lr']
    duree = time.time() - t0
    
    print(f"\n✅ FIN EP {epoch+1} | Time: {duree:.0f}s | F1 Val: {val_f1:.4f} | LR: {current_lr:.1e}")
    
    scheduler.step(val_f1)
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "livrable_model_resnet_final_clean.pth"))
        
        meta_data = {
            "model_name": "ResNet50 Robust Extended Final", 
            "score_f1": val_f1,
            "class_mapping": {int(i): str(c) for i, c in enumerate(le.classes_)}
        }
        with open(os.path.join(OUTPUT_DIR, "livrable_resnet_reborn_metadata.json"), 'w') as f:
            json.dump(meta_data, f, indent=4)
            
        print(f"   💾 NOUVEAU RECORD : {val_f1:.4f}")

print(f"🏆 Marathon terminé. Score final : {best_f1:.4f}")

🚀 RELANCE TOTALE (EXTENSION 30 EPOQUES) SUR : cuda
📂 Rechargement du Dataset...
🔧 Reconstruction de l'architecture...
📥 Chargement de ton champion (Epoque 15)...
✅ Poids chargés avec succès ! On reprend l'entraînement.
🔥 Démarrage Extension (Epoques 16 -> 45) | LR Fixé à: 1.0e-05
   ⏳ Ep 16 | Batch 2100 | Loss Train: 0.5318
✅ FIN EP 16 | Time: 625s | F1 Val: 0.6617 | LR: 1.0e-05
   💾 NOUVEAU RECORD : 0.6617
   ⏳ Ep 17 | Batch 2100 | Loss Train: 0.4231
✅ FIN EP 17 | Time: 577s | F1 Val: 0.6618 | LR: 1.0e-05
   💾 NOUVEAU RECORD : 0.6618
   ⏳ Ep 18 | Batch 2100 | Loss Train: 0.2502
✅ FIN EP 18 | Time: 562s | F1 Val: 0.6608 | LR: 1.0e-05
   ⏳ Ep 19 | Batch 2100 | Loss Train: 0.4842
✅ FIN EP 19 | Time: 565s | F1 Val: 0.6603 | LR: 1.0e-06
   ⏳ Ep 20 | Batch 2100 | Loss Train: 0.5718
✅ FIN EP 20 | Time: 565s | F1 Val: 0.6605 | LR: 1.0e-06
   ⏳ Ep 21 | Batch 2100 | Loss Train: 0.5222
✅ FIN EP 21 | Time: 566s | F1 Val: 0.6604 | LR: 1.0e-07
   ⏳ Ep 22 | Batch 2100 | Loss Train: 0.2447
✅ FIN EP 2

KeyboardInterrupt: 

In [3]:
# mon modele est actuellement bloque et ne progresse plus du tout
# je tente une approche pro pour casser ce plafond de verre
# j active le lissage des etiquettes pour eviter que le modele soit trop arrogant
# cela l aide a mieux comprendre les images difficiles sans apprendre par coeur
# je change aussi le rythme d apprentissage pour qu il fasse des vagues
# cela va secouer le modele et le forcer a trouver de meilleures solutions
# c est une technique tres efficace pour gagner les derniers points de precision

In [4]:
import os
import time
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

# --- CONFIGURATION ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
IMG_DIR = r"C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_train"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = torch.amp.GradScaler('cuda')

print(f"⚡ OPÉRATION 'BREAKTHROUGH' (LABEL SMOOTHING) SUR : {DEVICE}")

# --- RECONSTRUCTION DATA (Comme avant) ---
csv_path = os.path.join(PROJECT_ROOT, "data", "raw")
df_x = pd.read_csv(os.path.join(csv_path, "X_train_update.csv"), index_col=0)
df_y = pd.read_csv(os.path.join(csv_path, "Y_train_CVw08PX.csv"), index_col=0)
df = pd.merge(df_x, df_y, left_index=True, right_index=True)
df['path'] = df.apply(lambda x: os.path.join(IMG_DIR, f"image_{x['imageid']}_product_{x['productid']}.jpg"), axis=1)

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['prdtypecode'])
NUM_CLASSES = len(le.classes_)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)

class RobustDataset(Dataset):
    def __init__(self, df, mode='train'):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform_train = transforms.Compose([
            transforms.Resize((256, 256)),      
            transforms.RandomCrop(224),         
            transforms.RandomHorizontalFlip(p=0.5), 
            transforms.RandomRotation(15),      
            transforms.ColorJitter(brightness=0.3, contrast=0.3), # Augmenté un peu
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.transform_val = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try: img = Image.open(row['path']).convert("RGB")
        except: img = Image.new('RGB', (224, 224), (0, 0, 0))
        if self.mode == 'train': return self.transform_train(img), torch.tensor(row['label_encoded'], dtype=torch.long)
        else: return self.transform_val(img), torch.tensor(row['label_encoded'], dtype=torch.long)

train_loader = DataLoader(RobustDataset(train_df, 'train'), batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(RobustDataset(val_df, 'val'), batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# --- MODELE & CHARGEMENT ---
resnet = models.resnet50(weights=None)
class LegendMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048), nn.BatchNorm1d(2048), nn.GELU(), nn.Dropout(0.4), # Dropout 0.4 plus fort
            nn.Linear(2048, 1024), nn.BatchNorm1d(1024), nn.GELU(), nn.Dropout(0.4),
            nn.Linear(1024, 512),  nn.BatchNorm1d(512),  nn.GELU(), nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    def forward(self, x): return self.net(x)

resnet.fc = LegendMLP(2048, NUM_CLASSES)
model = resnet.to(DEVICE)

# On recharge le poids bloqué à 66% pour ne pas tout perdre
try:
    model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "livrable_model_resnet_final_clean.pth")))
    print("✅ Poids rechargés. On va les secouer.")
except:
    print("⚠️ Pas de poids trouvés, on part de zéro (ce sera long).")

# --- 🧪 L'ARME SECRÈTE : LABEL SMOOTHING + COSINE ANNEALING ---

# 1. Label Smoothing : On tolère l'incertitude (0.1)
# Au lieu de viser [0, 1, 0], on vise [0.05, 0.95, 0.05]
criterion = nn.CrossEntropyLoss(label_smoothing=0.1) 

# 2. Reset de l'optimiseur
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=2e-2) # Weight decay plus fort

# 3. Scheduler Cosine : On fait 3 cycles de restart(idée d'un coeur qu'on redémmarre et les vrais résulstats sont toutes les 5 épochs)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2, eta_min=1e-6)

print("🔥 DÉMARRAGE PROTOCOLE 'COSINE RESTART' (Optimisation SOTA)...")
best_f1 = 0.66 # On veut battre ça

for epoch in range(15): # On lance pour 15 époques de test (Cycle 1 + début Cycle 2)
    model.train()
    loss_ep = 0.0
    t0 = time.time()
    
    for i, (imgs, lbls) in enumerate(train_loader):
        imgs, lbls = imgs.to(DEVICE, non_blocking=True), lbls.to(DEVICE, non_blocking=True)
        
        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):
            out = model(imgs)
            loss = criterion(out, lbls)
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loss_ep += loss.item()
        
        # Le scheduler Cosine fonctionne par BATCH, pas par époque (souvent mieux)
        scheduler.step(epoch + i / len(train_loader))

        if i % 100 == 0:
            print(f"   ⏳ Ep {epoch+1} | Batch {i} | Loss: {loss.item():.4f} | LR: {optimizer.param_groups[0]['lr']:.2e}", end="\r")

    # Validation
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for imgs, lbls in val_loader:
            imgs = imgs.to(DEVICE, non_blocking=True)
            with torch.amp.autocast('cuda'):
                out = model(imgs)
            _, p = torch.max(out, 1)
            preds.extend(p.cpu().numpy())
            targets.extend(lbls.cpu().numpy())
    
    val_f1 = f1_score(targets, preds, average='weighted')
    duree = time.time() - t0
    
    # Indicateur visuel
    res = "😐"
    if val_f1 > best_f1: res = "🚀 RECORD"
    
    print(f"\n✅ FIN EP {epoch+1} | Time: {duree:.0f}s | F1 Val: {val_f1:.4f} | {res}")
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "livrable_model_resnet_cosine_best.pth"))

⚡ OPÉRATION 'BREAKTHROUGH' (LABEL SMOOTHING) SUR : cuda
✅ Poids rechargés. On va les secouer.
🔥 DÉMARRAGE PROTOCOLE 'COSINE RESTART' (Optimisation SOTA)...
   ⏳ Ep 1 | Batch 2100 | Loss: 1.5736 | LR: 9.07e-05
✅ FIN EP 1 | Time: 599s | F1 Val: 0.6430 | 😐
   ⏳ Ep 2 | Batch 2100 | Loss: 0.9871 | LR: 6.61e-05
✅ FIN EP 2 | Time: 573s | F1 Val: 0.6505 | 😐
   ⏳ Ep 3 | Batch 2100 | Loss: 1.2467 | LR: 3.55e-05
✅ FIN EP 3 | Time: 578s | F1 Val: 0.6541 | 😐
   ⏳ Ep 4 | Batch 2100 | Loss: 0.8575 | LR: 1.07e-05
✅ FIN EP 4 | Time: 576s | F1 Val: 0.6587 | 😐
   ⏳ Ep 5 | Batch 2100 | Loss: 1.1364 | LR: 1.00e-06
✅ FIN EP 5 | Time: 582s | F1 Val: 0.6602 | 🚀 RECORD
   ⏳ Ep 6 | Batch 1600 | Loss: 1.5174 | LR: 9.86e-05


KeyboardInterrupt

